# Records DB File

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

# import_raw_data_from_file(rawfilepath="../data/tmp/debug-scopus.csv", rawfiletype="scopus", dirpath="../data/processed/debug")
# process_records_file(dirpath="../data/processed/debug")

# Data Cleaning

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

# clean_institution_fields(dirpath="../data/processed/debug")
# clean_keywords_fields(dirpath="../data/processed/debug")

# apply_thesaurus(dirpath_or_records="../data/processed/debug", thesaurus_filepath="../data/processed/debug/keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus")
# create_thesaurus(dirpath_or_records="../data/processed/debug", column="keywords", thesaurus_filepath="../data/processed/debug/test_thesaurus.txt")

# Basic Analysis

In [2]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

# bradford_plot(dirpath_or_records="../data/processed/debug")
# core_authors(dirpath_or_records="../data/processed/debug")
# core_sources(dirpath_or_records="../data/processed/debug")
# coverage(dirpath_or_records="../data/processed/debug")
# lotka_plot(dirpath_or_records="../data/processed/debug")
# summary(dirpath_or_records="../data/processed/debug")
# most_cited_documents(dirpath_or_records="../data/processed/debug")
# terms_table(dirpath_or_records="../data/processed/debug", column='authors')
# worldmap(dirpath_or_records="../data/processed/debug", metric='num_records')
# time_table(dirpath_or_records="../data/processed/debug")


# TermAnalyzer(dirpath_or_records="../data/processed/debug", column='countries', top_by='num_records').table_
# TimeAnalyzer(dirpath_or_records="../data/processed/debug").barh(column='num_documents', cmap="Blues")
# worldmap(count_records_by_term(dirpath_or_records="../data/processed/debug", column="countries"))


# BigraphAnalyzer(dirpath_or_records="../data/processed/debug", column='countries', by='countries').table_

,num_records,global_citations,local_citations
Vanfretti L,6,45,0
Baudette M,5,44,0
Nakano A,4,15,0
Schwarze R,4,7,0
Vashishta P,4,15,0
...,...,...,...
Oren R,1,30,0
Orović I,1,0,0
Orozco M,1,16,0
Orozco-Arroyave JR,1,0,0


In [ ]:





# apply_thesaurus(directory_or_records="../data/processed/debug", thesaurus_filepath="../data/processed/debug/keywords.txt", input_column="author_keywords", output_column="author_keywords_thesaurus")
# association_index(co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords"), association='jaccard')

# co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords", by='author_keywords')
# co_occurrence_matrix(directory_or_records="../data/processed/debug", column="author_keywords", by='countries')



# create_thesaurus(directory_or_records="../data/processed/debug", column="keywords", thesaurus_filepath="../data/processed/debug/test_thesaurus.txt")
# impact_analysis(directory_or_records="../data/processed/debug", column="authors")
# tf_matrix(directory_or_records="../data/processed/debug", column="author_keywords")
# tfidf_matrix(tf_matrix(directory_or_records="../data/processed/debug", column="author_keywords"))

# collaboration_analysis(directory_or_records="../data/processed/debug", column="authors")

In [ ]:
import pandas as pd

pd.read_csv("../data/processed/debug/records.csv").author_keywords_thesaurus